In [1]:
from pyspark.sql import SparkSession, functions as F,pandas as ps

#import the pyspark module
import pyspark

#import the col function
from pyspark.sql.functions import col


# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/02 23:44:56 WARN Utils: Your hostname, guozixuandeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.47 instead (on interface en0)
22/09/02 23:44:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/02 23:44:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_bus = spark.read.format("csv").load("../data/raw/bus_stop.csv",header=True)
df_train = spark.read.format("csv").load("../data/raw/train_station.csv",header=True)
df_tram = spark.read.format("csv").load("../data/raw/tram_stop.csv",header=True)

In [3]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import geopy
import pandas as pd

def get_zipcode(df, geolocator, lat_field, lon_field):
    try:
        location = geolocator.reverse((df[lat_field], df[lon_field]))
        return location.raw['address']['postcode']
    except (AttributeError, KeyError, ValueError):
        return None
    
geolocator = geopy.Nominatim(user_agent='zixguo@student.unimelb.edu.au') #My OpenMap username



In [5]:
zipcodes_tram = df_tram.toPandas().apply(
    get_zipcode, axis=1, geolocator=geolocator, 
    lat_field='LATITUDE', lon_field='LONGITUDE')


In [16]:
new_df_tram = df_tram.toPandas().assign(postcode = zipcodes_tram)
new_df_tram

,FID,OBJECTID,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTES_USING_STOP,SHAPE,SE_ANNO_CAD_DATA,postcode
0,PTV_METRO_TRAM_STOP.fid-38574575_182f3aaa27b_718b,418,18079,38-Jacobs Reserve/Melville Rd (Brunswick West),-37.7566,144.9454,1,58,POINT (-37.75662297 144.94538484),None,3055
1,PTV_METRO_TRAM_STOP.fid-38574575_182f3aaa27b_718c,419,18080,39-Mattingley Cres/Melville Rd (Brunswick West),-37.7548,144.9457,1,58,POINT (-37.75480002 144.94570812),None,3055
2,PTV_METRO_TRAM_STOP.fid-38574575_182f3aaa27b_718d,420,18081,40-Moreland Rd/Melville Rd (Brunswick West),-37.7532,144.9459,1,58,POINT (-37.75316301 144.94590108),None,3055
3,PTV_METRO_TRAM_STOP.fid-38574575_182f3aaa27b_718e,421,18082,41-Woodlands Ave/Melville Rd (Pascoe Vale South),-37.7493,144.9466,1,58,POINT (-37.74931704 144.94655196),None,3044
4,PTV_METRO_TRAM_STOP.fid-38574575_182f3aaa27b_718f,422,18083,42-Reynard St/Melville Rd (Pascoe Vale South),-37.7464,144.9446,1,58,POINT (-37.74639897 144.94455612),None,3044
...,...,...,...,...,...,...,...,...,...,...,...
1660,PTV_METRO_TRAM_STOP.fid-38574575_182f3aaa27b_7807,1108,19687,10-Bourke Street Mall/Swanston St (Melbourne C...,-37.8128,144.9652,1,"16,3/3a,5,6,64,67,72,1",POINT (-37.81275903 144.96518484),None,3000
1661,PTV_METRO_TRAM_STOP.fid-38574575_182f3aaa27b_7808,1109,19689,8-Melbourne Central Station/Swanston St (Melbo...,-37.8099,144.9639,1,"16,3/3a,5,6,64,67,72,1",POINT (-37.80988902 144.96388992),None,3000
1662,PTV_METRO_TRAM_STOP.fid-38574575_182f3aaa27b_7809,1425,6044,27-Westgarth St/High St (Northcote),-37.7802,144.9968,1,86,POINT (-37.78024698 144.99684792),None,3070
1663,PTV_METRO_TRAM_STOP.fid-38574575_182f3aaa27b_780a,1426,6045,26-Walker St/High St (Northcote),-37.7836,144.9962,1,86,POINT (-37.78356996 144.99616716),None,3070


In [7]:
zipcodes_train = df_train.toPandas().apply(
    get_zipcode, axis=1, geolocator=geolocator, 
    lat_field='LATITUDE', lon_field='LONGITUDE')


In [15]:
new_df_train = df_train.toPandas().assign(postcode = zipcodes_train)
new_df_train

,FID,OBJECTID,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTES_USING_STOP,SHAPE,SE_ANNO_CAD_DATA,postcode
0,PTV_METRO_TRAIN_STATION.fid-38574575_182f3aaa2...,146,19970,Royal Park Railway Station (Parkville),-37.7812,144.9523,1,Upfield,POINT (-37.78119297 144.95230116),None,3052
1,PTV_METRO_TRAIN_STATION.fid-38574575_182f3aaa2...,147,19971,Flemington Bridge Railway Station (North Melbo...,-37.7881,144.9393,1,Upfield,POINT (-37.78813998 144.93932316),None,3031
2,PTV_METRO_TRAIN_STATION.fid-38574575_182f3aaa2...,148,19972,Macaulay Railway Station (North Melbourne),-37.7943,144.9362,1,Upfield,POINT (-37.794267 144.93616596),None,3031
3,PTV_METRO_TRAIN_STATION.fid-38574575_182f3aaa2...,149,19973,North Melbourne Railway Station (West Melbourne),-37.8074,144.9426,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (-37.80741897 144.94257),None,3003
4,PTV_METRO_TRAIN_STATION.fid-38574575_182f3aaa2...,150,19974,Clifton Hill Railway Station (Clifton Hill),-37.7887,144.9954,1,"Mernda,Hurstbridge",POINT (-37.78865703 144.99541692),None,3070
...,...,...,...,...,...,...,...,...,...,...,...
215,PTV_METRO_TRAIN_STATION.fid-38574575_182f3aaa2...,141,19965,Coburg Railway Station (Coburg),-37.7423,144.9633,1,Upfield,POINT (-37.74234501 144.96333588),None,3058
216,PTV_METRO_TRAIN_STATION.fid-38574575_182f3aaa2...,142,19966,Moreland Railway Station (Coburg),-37.7545,144.9618,1,Upfield,POINT (-37.75448502 144.96182316),None,3058
217,PTV_METRO_TRAIN_STATION.fid-38574575_182f3aaa2...,143,19967,Anstey Railway Station (Brunswick),-37.7612,144.9607,1,Upfield,POINT (-37.76124204 144.96068412),None,3056
218,PTV_METRO_TRAIN_STATION.fid-38574575_182f3aaa2...,144,19968,Brunswick Railway Station (Brunswick),-37.7677,144.9596,1,Upfield,POINT (-37.76772096 144.95958684),None,3056


In [ ]:
zipcodes_bus = df_bus.toPandas().apply(
    get_zipcode, axis=1, geolocator=geolocator, 
    lat_field='LATITUDE', lon_field='LONGITUDE')


In [ ]:
zipcodes_bus

In [ ]:
new_df_bus = df_bus.toPandas().assign(postcode = zipcodes_bus)